### Necessary Imports

In [45]:
import sys 
## https://stackoverflow.com/questions/12257747/permanently-adding-a-file-path-to-sys-path-in-python
sys.path.append("/Users/lohyikuang/Downloads/personal_projects/the_heron_project/audrii/kedro-environment/env/kedro-environment/lib/python3.7/site-packages")

In [ ]:
import pyfolio
import pandas as pd
from api.assets import get_ticker_historical_data

### Simulate data

In [ ]:
simulated_data = {
    "GPS": [{"_id": "1", "action": "Long", "units": "20", "date": "2022-02-03", "price": "14.57", "simulatedFee": ""},
            {"_id": "2", "action": "Short", "units": "20", "date": "2022-04-04", "price": "14.21", "simulatedFee": ""},
            {"_id": "3", "action": "Long", "units": "25", "date": "2022-06-13", "price": "9.03", "simulatedFee": "0.2"}],
    "IAU": [{"_id": "1", "action": "Long", "units": "10", "date": "2022-03-02", "price": "36.61", "simulatedFee": ""},
            {"_id": "2", "action": "Short", "units": "10", "date": "2022-03-08", "price": "38.98", "simulatedFee": ""},
            {"_id": "3", "action": "Long", "units": "5", "date": "2022-03-15", "price": "36.45", "simulatedFee": "0.1"},
            {"_id": "4", "action": "Long", "units": "5", "date": "2022-04-13", "price": "37.62", "simulatedFee": "0.23"}],
    "M": [{"_id": "1", "action": "Long", "units": "55", "date": "2022-05-24", "price": "17.61", "simulatedFee": ""}],
    "NFLX": [{"_id": "1", "action": "Long", "units": "2", "date": "2022-01-19", "price": "515.86", "simulatedFee": ""}],
    "STNG": [{"_id": "1", "action": "Long", "units": "50", "date": "2022-04-24", "price": "20.60", "simulatedFee": "0.26"},
             {"_id": "2", "action": "Long", "units": "20", "date": "2022-05-13", "price": "28.35", "simulatedFee": "0.29"},
             {"_id": "3", "action": "Short", "units": "45", "date": "2022-05-24", "price": "30.46", "simulatedFee": "0.31"},
             {"_id": "4", "action": "Long", "units": "20", "date": "2022-06-02", "price": "35.19", "simulatedFee": "0.35"}]
}

In [ ]:
df = get_ticker_historical_data("GPS")

### Get our universe of symbols

In [ ]:
from api.mongodb import mongodb_client
tickers = mongodb_client.get_all_items("ticker_symbols")

In [ ]:
tickers = pd.DataFrame(tickers)
equity = tickers[tickers.quote_type == "Equity"]
etfs = tickers[tickers.quote_type == "ETF"]